In [1]:
import os

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

os.environ["WANDB_API_KEY"] = "KEY"
os.environ["WANDB_MODE"] = 'offline'
from itertools import combinations

import clip
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import tqdm

from einops.layers.torch import Rearrange, Reduce

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
import random
import csv
from torch import Tensor
import itertools
import math
import re
import numpy as np
import argparse
from torch import nn
from torch.optim import AdamW

In [2]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
import open_clip
device = 'cuda'
model_type = 'ViT-H-14'

vlmodel, preprocess_train, feature_extractor = open_clip.create_model_and_transforms(
    'ViT-H-14', pretrained = '/root/autodl-tmp/open_clip_pytorch_model.bin', device = device
)


/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_ep_data(EP_folderpath, image_folderpath, vlmodel, preprocess_train, batch_size=20):
    EP_segmentation_list = os.listdir(EP_folderpath)
    print(f'Total Dataset Number: {len(EP_segmentation_list)}')
    
    # Initialize dictionaries with date as key
    EP_data_dict = defaultdict(list)
    images_dict = defaultdict(list)
    labels_dict = defaultdict(list)
    
    for EP_data_dir in EP_segmentation_list:
        # Extract date from filename (format: date_image_trial.npy)
        date_str = EP_data_dir.split("_")[0]
        
        # Load data and process as before
        EP_data = np.load(os.path.join(EP_folderpath, EP_data_dir))
        image_label = EP_data_dir.split("_")[1]
        image = Image.open(os.path.join(image_folderpath, f"{image_label}.jpg")).convert("L")
        image_label = int(image_label) - 1
        
        # Append to dictionaries
        EP_data_dict[date_str].append(EP_data)
        images_dict[date_str].append(image)
        labels_dict[date_str].append(image_label)

    # Function to process images in batches
    def image_encoder(images, vlmodel, preprocess, batch_size):
        features = []
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i+batch_size]
            batch_input = torch.stack([preprocess(img) for img in batch_images]).to('cuda')
            with torch.no_grad():
                batch_features = vlmodel.encode_image(batch_input)
                batch_features /= batch_features.norm(dim=-1, keepdim=True)
            features.append(batch_features.cpu())
        return torch.cat(features)
    
    # Process features for each date separately
    features_dict = {}
    for date_str in images_dict:
        image_features = image_encoder(images_dict[date_str], vlmodel, preprocess_train, batch_size)
        features_dict[date_str] = image_features
    
    return EP_data_dict, images_dict, labels_dict, features_dict

class EPDataset(Dataset):
    def __init__(self, EP_data, images, labels, features, neuron_mask):  
        self.EP_data = EP_data
        self.images = images
        self.labels = labels
        self.features = features
        self.max_pool = nn.MaxPool1d(kernel_size=5, stride=5)
        self.neuron_mask = neuron_mask 

    def __len__(self):
        return len(self.EP_data)
        
    def __getitem__(self, idx):
        EP_sample = self.EP_data[idx].T
        EP_filtered = EP_sample[:, self.neuron_mask]
        label = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        feature = self.features[idx]
        
        EP_tensor = torch.tensor(EP_filtered, dtype=torch.float32) 
        
        return EP_tensor, label, feature

EP_folderpath = "/root/autodl-tmp/data_neuron_loss_month6"
image_folderpath = "/root/visual_decode/NaturalImages_new_2"
vlmodel = vlmodel  
preprocess_train = preprocess_train  



In [5]:
from collections import defaultdict

EP_data_dict, images_dict, labels_dict, features_dict = load_ep_data(
    EP_folderpath, 
    image_folderpath,
    vlmodel,
    preprocess_train,
    batch_size=40
)

Total Dataset Number: 17924


In [16]:
class ModelConfig:
    def __init__(self,
                 input_neuron=25,        
                 time_bins=20,          
                 d_model = 150,          
                 nhead=10,                
                 num_transformer_layers=1, 
                 conv_channels=64,      
                 num_conv_blocks=3,      
                 num_classes=117,        
                 residual_dims=[256, 512, 1024], 
                 use_positional_encoding=True,  
                 dim_feedforward_ratio=4,      
                 activation='relu',
                 use_neuron_masking=True,  
                 mask_ratio=0,
                 mask_replacement='zero'):
        
        # Transformer 
        self.transformer = {
            'd_model': d_model,
            'nhead': nhead,
            'num_layers': num_transformer_layers,
            'dim_feedforward': d_model * dim_feedforward_ratio,
            'activation': activation
        }
        
        # cnn
        self.convolution = {
            'channels': conv_channels,
            'num_blocks': num_conv_blocks,
            'kernel_size': (3, 3),
            'pool_size': (2, 2)
        }
        
        # resnet
        self.residual = {
            'dims': residual_dims,
            'skip_connection': True
        }
        
        self.masking = {
            'enabled': use_neuron_masking,
            'ratio': mask_ratio,
            'replacement': mask_replacement
        }

        self.input_dim = input_neuron
        self.time_steps = time_bins
        self.num_classes = num_classes
        self.positional_encoding = use_positional_encoding
        self.lr = 2e-4
        self.epochs = 100

In [17]:
import torch
import torch.nn as nn
import math

def detect_lost_neurons(ep_data_list):
    if not ep_data_list:
        return np.zeros(25, dtype=bool)
    
    all_data = np.stack(ep_data_list)
    neuron_activity = np.sum(np.abs(all_data), axis=(0, 2))
    return neuron_activity == 0
    
class NeuronMasker(nn.Module):
    def __init__(self, mask_ratio=0.15, replacement='random'):
        super().__init__()
        self.mask_ratio = mask_ratio
        self.replacement = replacement
        
    def forward(self, x):
        if self.training:
            if x is None:
                raise ValueError("Input tensor x is None")
                
            batch_size, seq_len, feat_dim = x.shape
            mask = torch.rand(batch_size, 1, feat_dim, device=x.device) < self.mask_ratio
            mask = mask.expand_as(x)
            
            if self.replacement == 'zero':
                x_masked = x.masked_fill(mask, 0)
            elif self.replacement == 'random':
                random_values = torch.randn_like(x) * 0.02
                x_masked = x.masked_scatter(mask, random_values)
            else:
                raise ValueError(f"Invalid replacement: {self.replacement}")
            
            return x_masked 
        else:
            return x
        
class ResidualLinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.norm = nn.LayerNorm(output_dim)
        self.activation = nn.GELU()
        self.downsample = nn.Linear(input_dim, output_dim) if input_dim != output_dim else nn.Identity()

    def forward(self, x):
        residual = self.downsample(x)
        x = self.linear(x)
        x = self.norm(x)
        x = self.activation(x)
        return x + residual

class TimeTransformerConvModel(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        
        self.input_proj = nn.Linear(config.input_dim, config.transformer['d_model'])
        self.pos_encoder = PositionalEncoding(config.transformer['d_model']) if config.positional_encoding else nn.Identity()
        
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=config.transformer['d_model'],
            nhead=config.transformer['nhead'],
            dim_feedforward=config.transformer['dim_feedforward'],
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(transformer_layer, config.transformer['num_layers'])
        
        self.conv_blocks = nn.Sequential()
        in_channels = 1
        for _ in range(config.convolution['num_blocks']):
            self.conv_blocks.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, config.convolution['channels'], 
                            kernel_size=config.convolution['kernel_size'], padding='same'),
                    nn.BatchNorm2d(config.convolution['channels']),
                    nn.ELU(),
                    nn.MaxPool2d(kernel_size=config.convolution['pool_size'])
                )
            )
            in_channels = config.convolution['channels']
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(config.convolution['channels'], config.num_classes)
        
        self.residual_layers = nn.Sequential()
        current_dim = config.convolution['channels']
        for dim in config.residual['dims']:
            self.residual_layers.append(ResidualLinearBlock(current_dim, dim))
            current_dim = dim
        if current_dim != 1024:
            self.residual_layers.append(nn.Linear(current_dim, 1024))
            self.residual_layers.append(nn.LayerNorm(1024))


        self.masker = NeuronMasker(
            mask_ratio=self.config.masking['ratio'],
            replacement=self.config.masking['replacement']
        )

    def forward(self, x):
        x = self.masker(x)  # [B, T, D]
        #print(x)
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        
        x = x.unsqueeze(1)
        x = self.conv_blocks(x)
        x = self.adaptive_pool(x)
        x = x.flatten(1)
        
        logits = self.classifier(x)
        features = self.residual_layers(x)
        
        return logits, features

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1)]
        return x

In [18]:
class MultitaskLoss(nn.Module):
    def __init__(self, alpha=0.3, temp=0.07):
        super().__init__()
        self.alpha = alpha      # 分类损失权重
        self.temp = temp
        self.ce_loss = nn.CrossEntropyLoss()
        self.temp = temp
        
        self.ce_loss = nn.CrossEntropyLoss()
    
    def contrastive_loss(self, h_neuro, h_img):
        h_neuro = F.normalize(h_neuro, dim=1) + 1e-10
        h_img = F.normalize(h_img, dim=1) + 1e-10
        
        logits_ab = torch.matmul(h_neuro, h_img.T) / self.temp
        logits_ba = torch.matmul(h_img, h_neuro.T) / self.temp
        
        labels = torch.arange(h_neuro.size(0), device=h_neuro.device)
        loss_ab = F.cross_entropy(logits_ab, labels)
        loss_ba = F.cross_entropy(logits_ba, labels)
        
        return (loss_ab + loss_ba) / 2
    
    def forward(self, logits, labels, img_feature, features):
        loss_cls = self.ce_loss(logits, labels)
        loss_cont = self.contrastive_loss(features, img_feature)
        total_loss = self.alpha * loss_cls + (1 - self.alpha) * loss_cont
        return total_loss

In [19]:
def train_model(model, dataloader, optimizer, device, criterion, config):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (neuro, labels, img_feature) in enumerate(dataloader):
        neuro = neuro.to(device)
        labels = labels.to(device)
        img_feature = img_feature.to(device)

        optimizer.zero_grad()
        
        logits, features = model(neuro)
        loss = criterion(logits, labels, img_feature, features)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), 
            max_norm=3.0,                   
            norm_type=2.0                   
        )
        optimizer.step()
        
        # 统计指标
        total_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
    train_loss = total_loss / len(dataloader)
    train_accuracy = correct / total
    return train_loss, train_accuracy

@torch.no_grad()
def evaluate_model(model, dataloader, device, criterion, config, image_cluster):
    model.eval()
    total_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    correct_2way = 0
    correct_4way = 0
    correct_10way = 0
    total = 0
    
    # 从 DataFrame 创建聚类映射字典
    cluster_2_map = torch.tensor(image_cluster["2_cluster"].values, device=device)
    cluster_4_map = torch.tensor(image_cluster["4_cluster"].values, device=device)
    cluster_10_map = torch.tensor(image_cluster["10_cluster"].values, device=device)
    
    for neuro, labels, img_feature in dataloader:
        neuro = neuro.to(device)
        labels = labels.to(device)
        img_feature = img_feature.to(device)
        
        logits, features = model(neuro)
        
        loss = criterion(logits, labels, img_feature, features)
        total_loss += loss.item()
        
        # （Top-1 和 Top-5）
        _, predicted_top1 = torch.max(logits, 1)
        correct_top1 += (predicted_top1 == labels).sum().item()
        _, predicted_top5 = logits.topk(5, dim=1)
        correct_top5 += torch.sum(predicted_top5.eq(labels.view(-1, 1))).item()
        
        # (2-way, 4-way, 10-way)
        cluster_2_pred = cluster_2_map[predicted_top1]
        cluster_4_pred = cluster_4_map[predicted_top1]
        cluster_10_pred = cluster_10_map[predicted_top1]
        
        cluster_2_true = cluster_2_map[labels]
        cluster_4_true = cluster_4_map[labels]
        cluster_10_true = cluster_10_map[labels]
        
        correct_2way += (cluster_2_pred == cluster_2_true).sum().item()
        correct_4way += (cluster_4_pred == cluster_4_true).sum().item()
        correct_10way += (cluster_10_pred == cluster_10_true).sum().item()
        
        total += labels.size(0)
    
    test_loss = total_loss / len(dataloader)
    test_accuracy = correct_top1 / total
    top5_accuracy = correct_top5 / total
    accuracy_2way = correct_2way / total
    accuracy_4way = correct_4way / total
    accuracy_10way = correct_10way / total
    
    return test_loss, test_accuracy, top5_accuracy, accuracy_2way, accuracy_4way, accuracy_10way

In [33]:
def main_train_loop(config, model, train_loader, test_loader, device, image_cluster):
    optimizer = AdamW(model.parameters(), lr=config.lr)
    criterion = MultitaskLoss(alpha=0.7, temp=0.07)
    
    train_losses, train_accs = [], []
    test_losses, test_accs, test_top5, acc_2way, acc_4way, acc_10way = [], [], [], [], [], []
    best_acc = 0.0
    
    for epoch in range(config.epochs):
        train_loss, train_acc = train_model(
            model=model,
            dataloader=train_loader,
            optimizer=optimizer,
            device=device,
            criterion=criterion,
            config=config
        )
        
        test_loss, test_acc, top5_acc, accuracy_2way, accuracy_4way, accuracy_10way = evaluate_model(
            model=model,
            dataloader=test_loader,
            device=device,
            criterion=criterion,
            config=config,
            image_cluster = image_cluster
        )
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        test_top5.append(top5_acc)
        acc_2way.append(accuracy_2way)
        acc_4way.append(accuracy_4way)
        acc_10way.append(accuracy_10way)
        
        
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), "best_model.pth")
            best_epoch = epoch
        
        # 打印日志
        print(f"Epoch {epoch+1}/{config.epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2%}")
        print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2%} | Top-5 Acc: {top5_acc:.2%}")
        print(f"2-way Acc: {accuracy_2way:.2%} | 4-way Acc: {accuracy_4way:.2%} | 10-way Acc: {accuracy_10way:.2%}")
        print("-" * 60)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train")
    plt.plot(test_losses, label="Test")
    plt.title("Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label="Train Acc")
    plt.plot(test_accs, label="Test Acc")
    plt.plot(test_top5, label="Test Top-5")
    plt.title("Accuracy Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    
    plt.tight_layout()
    plt.close()
    
    return {
        "best_test_acc": best_acc,
        "final_top5_acc": test_top5[-1],
        "train_history": {
            "loss": train_losses,
            "accuracy": train_accs
        },
        "test_history": {
            "loss": test_losses,
            "accuracy": test_accs,
            "top5_accuracy": test_top5,
            "acc_2way": acc_2way,
            "acc_4way": acc_4way,
            "acc_10way": acc_10way
        },
        "best_epoch": best_epoch
    }

In [34]:
date_order = ['021322', '022522', '031722', '042422', 
              '052422', '062422', '072322', '082322', 
              '092422', '102122', '112022', '122022', 
              #'012123', 
              '022223', '032123', '042323']



In [ ]:
for trial in range(1, 21):
    results_dict = {}
    neuron_mask = np.ones(25, dtype=bool)

    for slide in range(1, len(date_order)): 
        train_month = date_order[:slide]
        test_month = [date_order[slide]]
        
        EP_data_train = []
        images_train = []
        labels_train = []
        features_train = []
        
        for month in train_month:
            if month not in EP_data_dict:
                raise ValueError(f"Missing data for training month: {month}")
            month_EP = EP_data_dict[month]
            month_images = images_dict[month]
            month_labels = labels_dict[month]
            month_features = features_dict[month]
            
            EP_data_train.extend(month_EP)
            images_train.extend(month_images)
            labels_train.extend(month_labels)
            features_train.append(month_features)  
            
        features_train = torch.cat(features_train, dim=0) if features_train else torch.Tensor()

        EP_data_test = []
        images_test = []
        labels_test = []
        features_test = []
        
        for month in test_month:
            if month not in EP_data_dict:
                raise ValueError(f"Missing data for training month: {month}")
            month_EP = EP_data_dict[month]
            month_images = images_dict[month]
            month_labels = labels_dict[month]
            month_features = features_dict[month]
            
            EP_data_test.extend(month_EP)
            images_test.extend(month_images)
            labels_test.extend(month_labels)
            features_test.append(month_features)  

        features_test = torch.cat(features_test, dim=0) if features_test else torch.Tensor()

        if EP_data_test:
            lost_neurons = detect_lost_neurons(EP_data_test[: 20])
            neuron_mask[lost_neurons] = False

        print(f'Lossed Neuron Number for month {test_month}: {np.sum(lost_neurons!=0)}')
        train_dataset = EPDataset(
            EP_data_train, images_train, labels_train, features_train, neuron_mask
        )
        test_dataset = EPDataset(
            EP_data_test, images_test, labels_test, features_test, neuron_mask
        )
        
        torch.cuda.empty_cache()
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)

        current_input_neuron = sum(neuron_mask)
        config = ModelConfig(input_neuron=current_input_neuron)
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = TimeTransformerConvModel(config).to(device)

        image_cluster = pd.read_csv("image_cluster.csv")
        #print(f'Training on slide: {slide}, Trial: {trial}')
        results = main_train_loop(
            config=config,
            model=model,
            train_loader=train_loader,
            test_loader=test_loader,
            device=device,
            image_cluster = image_cluster
        )

        results_dict[slide] = results

    import pickle
    with open(f"/root/visual_decode/result_month_slide_neuron_loss_mask03/result_month_slide_neuron_loss_mask03_{trial}.pkl", "wb") as f:
        pickle.dump(results_dict, f)

    print(f"Results saved for Trial: {trial}")
    print("-" * 60)


Lossed Neuron Number for month ['022522']: 0
Lossed Neuron Number for month ['031722']: 1
Lossed Neuron Number for month ['042422']: 5
Lossed Neuron Number for month ['052422']: 5


In [35]:
neuron_mask = np.ones(25, dtype=bool)

for slide in [12]: 
    train_month = date_order[:slide]
    test_month = [date_order[slide]]
    
    EP_data_train = []
    images_train = []
    labels_train = []
    features_train = []
    
    for month in train_month:
        if month not in EP_data_dict:
            raise ValueError(f"Missing data for training month: {month}")
        month_EP = EP_data_dict[month]
        month_images = images_dict[month]
        month_labels = labels_dict[month]
        month_features = features_dict[month]
        
        EP_data_train.extend(month_EP)
        images_train.extend(month_images)
        labels_train.extend(month_labels)
        features_train.append(month_features)  
        
    features_train = torch.cat(features_train, dim=0) if features_train else torch.Tensor()

    EP_data_test = []
    images_test = []
    labels_test = []
    features_test = []
    
    for month in test_month:
        if month not in EP_data_dict:
            raise ValueError(f"Missing data for training month: {month}")
        month_EP = EP_data_dict[month]
        month_images = images_dict[month]
        month_labels = labels_dict[month]
        month_features = features_dict[month]
        
        EP_data_test.extend(month_EP)
        images_test.extend(month_images)
        labels_test.extend(month_labels)
        features_test.append(month_features)  

    features_test = torch.cat(features_test, dim=0) if features_test else torch.Tensor()

    if EP_data_test:
        lost_neurons = detect_lost_neurons(EP_data_test[: 20])
        neuron_mask[lost_neurons] = False

    print(f'Lossed Neuron Number for month {test_month}: {np.sum(lost_neurons!=0)}')
    train_dataset = EPDataset(
        EP_data_train, images_train, labels_train, features_train, neuron_mask
    )
    test_dataset = EPDataset(
        EP_data_test, images_test, labels_test, features_test, neuron_mask
    )
    
    torch.cuda.empty_cache()
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)

    current_input_neuron = sum(neuron_mask)
    config = ModelConfig(input_neuron=current_input_neuron)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TimeTransformerConvModel(config).to(device)

    image_cluster = pd.read_csv("image_cluster.csv")
    #print(f'Training on slide: {slide}, Trial: {trial}')
    results = main_train_loop(
        config=config,
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        device=device,
        image_cluster = image_cluster
    )

Lossed Neuron Number for month ['022223']: 6
Epoch 1/100
Train Loss: 4.5026 | Train Acc: 1.60%
Test Loss: 4.6531 | Test Acc: 0.83% | Top-5 Acc: 5.42%
2-way Acc: 48.07% | 4-way Acc: 23.81% | 10-way Acc: 11.21%
------------------------------------------------------------
Epoch 2/100
Train Loss: 4.1826 | Train Acc: 3.32%
Test Loss: 4.7362 | Test Acc: 0.92% | Top-5 Acc: 4.50%
2-way Acc: 50.64% | 4-way Acc: 25.64% | 10-way Acc: 12.41%
------------------------------------------------------------
Epoch 3/100
Train Loss: 3.9717 | Train Acc: 4.61%
Test Loss: 5.0050 | Test Acc: 1.65% | Top-5 Acc: 6.43%
2-way Acc: 53.58% | 4-way Acc: 28.49% | 10-way Acc: 11.03%
------------------------------------------------------------
Epoch 4/100
Train Loss: 3.8473 | Train Acc: 5.95%
Test Loss: 5.3531 | Test Acc: 1.38% | Top-5 Acc: 6.16%
2-way Acc: 51.47% | 4-way Acc: 27.30% | 10-way Acc: 8.46%
------------------------------------------------------------
Epoch 5/100
Train Loss: 3.7559 | Train Acc: 7.44%
Test L